In [99]:
import numpy as np
import pandas as pd
import sys
import os
import duckdb

In [100]:
sys.path.append('..')
from helper import *

setwd()
Objects = Objects()

In [84]:
# connection to duckdb
con = duckdb.connect(database="data/formatted/db_formatted.db", read_only=False)

In [91]:
con.close()

## select files to process

In [97]:
# files already processed:
processedFiles = []
with open('logs/processed_data_persistant.txt', "r") as f0:
    for line in f0:
        processedFiles.append(line.rstrip())
processedFiles

[]

In [ ]:
# files in the temporal zone:
for datasource in Objects:
    id = datasource["id"]
    print(id)
    print(os.listdir(f'data/landing_pers/{id}'))

In [96]:
# inputs:
# - Datasource: element of the Objects.json dictionary
# - Database: opened duckdb connection
# it stores the tables in the landing zone in Database
def ProcessCSV(Datasource, ProcessedFiles = [], LogWriter = ""):

    id = datasource["id"]
    # csv delimiter
    delim = datasource["delim"] if "delim" in datasource else ","
    # open database connection
    con = duckdb.connect(database=f"data/formatted/db_{id}.db", read_only=False)


    # files to process:
    files = [file for file in os.listdir(f'data/landing_pers/{id}') if not file in processedFiles]
    
    
    # TODO select only files that have not been yet processed
    for file in files:
        tablename = file.split(".")[0]
        con.execute(f"""
            CREATE TABLE test_{tablename}
            AS SELECT * FROM read_csv_auto(
                './data/landing_pers/{id}/{file}',
                HEADER = TRUE,
                DELIM = '{delim}'
            ) 
            """)
    
    # close database
    con.close()

In [ ]:
for datasource in Objects:

    id = datasource["id"]
    print(id)

    format = datasource["format"]
    
    if format == "csv":
        ProcessCSV(datasource, con)
    elif format == "xlsx":
        ProcessXLSX(datasource)
    else:
        print(f"Format {format} is not supported\nSupported formats: csv, xlsx")

In [89]:
import pandas as pd
def ProcessXLSX(Datasource):
    id = Datasource["id"]
    
    # TODO select only files that have not been yet processed
    for table in os.listdir(f'data/landing_pers/{id}'):
        pass
        #df = pd.read_excel(f'data/landing_pers/{id}/{table}')
        #display(df.head())

ProcessXLSX(Objects[0])

## useful queries to debug

In [62]:
con.execute("SHOW TABLES")
con.fetchall()

[('test',), ('test_ine_20221001_231336',)]

In [64]:
con.execute("DESCRIBE test_ine_20221001_231336")
con.fetchall()

[('Provincias', 'VARCHAR', 'YES', None, None, None),
 ('Grupos ECOICOP', 'VARCHAR', 'YES', None, None, None),
 ('Tipo de dato', 'VARCHAR', 'YES', None, None, None),
 ('Periodo', 'VARCHAR', 'YES', None, None, None),
 ('Total', 'VARCHAR', 'YES', None, None, None)]

In [68]:
con.execute("DROP TABLE test_ine_20221001_231336")

In [71]:
# con.execute()
con.description
con.fetchall()

[(686244,)]

In [72]:
con.execute("SELECT * FROM test LIMIT 10")
con.fetchall()

In [80]:
con.close()